In [ ]:
import sqlite3
import pandas as pd

In [53]:
query = """
    WITH ArtistGenres AS (
        SELECT 
            a.artist_id,
            GROUP_CONCAT(ag.genre_id, ',') AS genres
        FROM 
            r_artist_genre ag
        JOIN 
            r_track_artist a
        ON 
            a.artist_id = ag.artist_id
        GROUP BY 
            a.artist_id
    ),

    NumberedArtists AS (
        SELECT 
            track_id,
            artist_id,
            ROW_NUMBER() OVER (PARTITION BY track_id) AS rn
        FROM r_track_artist
    )
    
SELECT 
    t.id, 
    a.artist_id, 
    ag.genres,
    t.duration, 
    t.popularity, 
    af.acousticness, 
    af.danceability, 
    af.energy, 
    af.instrumentalness, 
    af.key,
    af.liveness, 
    af.loudness, 
    af.speechiness, 
    af.tempo, 
    af.time_signature, 
    af.valence
FROM 
    tracks t
LEFT JOIN 
    audio_features af
ON 
    t.audio_feature_id = af.id
LEFT JOIN 
    NumberedArtists a
ON 
    t.id = a.track_id
LEFT JOIN 
    ArtistGenres ag
ON 
    a.artist_id = ag.artist_id
where a.rn = 1
LIMIT 500;
"""

In [54]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('/Users/idotzhori/Downloads/spotify.sqlite')
# Get table names

# Fetch data from a specific table (replace 'your_table_name' with your table's name)
conn.text_factory = lambda b: b.decode(errors='ignore')
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the DataFrame
df.tail(50)

,id,artist_id,genres,duration,audio_feature_id,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
450,3FQyijPxxQZOD7Q5n9WJiD,4QKx7KohkWZAOkO4sI3GRx,"cartoon,cartoon,cartoon,cartoon,cartoon,cartoo...",206266,3FQyijPxxQZOD7Q5n9WJiD,44,0.727000,0.590,0.3400,0.000000,0,0.2330,-8.144000,0.2610,132.330994,4,0.5560
451,5bXGJIOZZ4TwcUpRYqf450,1FKfMww5N8dkbyo2sp9nBA,"bossa nova,bossa nova,bossa nova,bossa nova,bo...",228026,5bXGJIOZZ4TwcUpRYqf450,47,0.963000,0.527,0.1960,0.905000,7,0.0940,-17.663000,0.0339,65.869003,4,0.1800
452,5L2sEUGZoVeMzznTBADoJf,0FQUUcABeNbZZtWq8IRE3w,"norteno,norteno,norteno,norteno,norteno,norten...",171466,5L2sEUGZoVeMzznTBADoJf,9,0.258000,0.660,0.5580,0.000000,4,0.0694,-5.023000,0.0342,95.004997,4,0.7520
453,5nGKkJgYRtOw2tdIxmG7F3,24z2EWgF5nVUKnbJHd9ku2,"corrido,corrido,corrido,corrido,corrido,corrid...",224792,5nGKkJgYRtOw2tdIxmG7F3,30,0.663000,0.545,0.4060,0.000000,6,0.0365,-7.638000,0.0843,67.523003,4,0.9640
454,5U5A6xxLDhRK9D623CPJxW,7IIjgbSGJqA8mgiHz1nYl9,"bulgarian rock,bulgarian rock,bulgarian rock,b...",192191,5U5A6xxLDhRK9D623CPJxW,0,0.029900,0.602,0.6730,0.000005,7,0.1020,-11.586000,0.0396,143.192001,4,0.7730
455,5ZaouisGm3mMGXlY1SQ4FN,3JsMj0DEzyWc0VDlHuy9Bx,"album rock,album rock,album rock,album rock,al...",284533,5ZaouisGm3mMGXlY1SQ4FN,45,0.436000,0.419,0.5560,0.000002,5,0.2890,-7.798000,0.0272,104.439003,4,0.4790
456,628Btf2AY0aDYclqQ8VQ9d,1g8HCTiMwBtFtpRR9JXAZR,"classic j-rock,classic j-rock,classic j-rock,c...",300653,628Btf2AY0aDYclqQ8VQ9d,36,0.012100,0.698,0.6050,0.436000,9,0.1100,-10.546000,0.0420,98.041000,4,0.4450
457,6bwkndsWkJyGg7QvRIr00J,3S9VOVAAECM8pdNr5p7c2J,"classic schlager,classic schlager,classic schl...",158160,6bwkndsWkJyGg7QvRIr00J,0,0.503000,0.391,0.5200,0.000000,0,0.4740,-6.888000,0.0283,133.751007,4,0.7620
458,6MTEQh2ZHMqqbd5RPPvTCy,3Z02hBLubJxuFJfhacLSDc,"album rock,album rock,album rock,album rock,al...",243066,6MTEQh2ZHMqqbd5RPPvTCy,26,0.009460,0.508,0.6780,0.032500,2,0.2100,-8.677000,0.0317,103.002998,4,0.2330
459,6PlxB2XQWnEXkwYPpgzbtT,1FKfMww5N8dkbyo2sp9nBA,"bossa nova,bossa nova,bossa nova,bossa nova,bo...",161666,6PlxB2XQWnEXkwYPpgzbtT,32,0.188000,0.579,0.8180,0.000003,2,0.0917,-8.997000,0.0438,171.509995,4,0.8610


In [65]:
set((df['genres'][0].split(',')))

{'g funk', 'gangster rap', 'hip hop', 'pop rap', 'rap', 'west coast rap'}

In [66]:
# Connect to the SQLite database
conn = sqlite3.connect('/Users/idotzhori/Downloads/spotify.sqlite')
# Get table names

# Fetch data from a specific table (replace 'your_table_name' with your table's name)
conn.text_factory = lambda b: b.decode(errors='ignore')
df = pd.read_sql_query("""select * from r_artist_genre""", conn)

# Close the connection
conn.close()

# Display the DataFrame
df.head(50)

,genre_id,artist_id
0,detroit hip hop,4tujQJicOnuZRLiBFdp3Ou
1,g funk,4tujQJicOnuZRLiBFdp3Ou
2,gangster rap,4tujQJicOnuZRLiBFdp3Ou
3,hardcore hip hop,4tujQJicOnuZRLiBFdp3Ou
4,hip hop,4tujQJicOnuZRLiBFdp3Ou
5,pop rap,4tujQJicOnuZRLiBFdp3Ou
6,rap,4tujQJicOnuZRLiBFdp3Ou
7,west coast rap,4tujQJicOnuZRLiBFdp3Ou
8,gangster rap,2VX0o9LDIVmKIgpnwdJpOJ
9,hardcore hip hop,2VX0o9LDIVmKIgpnwdJpOJ


In [67]:
df.sort_values()

(487386, 2)

In [16]:
import sqlite3

conn = sqlite3.connect('/Users/idotzhori/Downloads/spotify.sqlite')
cursor = conn.cursor()

# Fetch all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

table_columns_dict = {}

# For each table, fetch its columns and store them in the dictionary
for table_name in tables:
    table_name = table_name[0]
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = [column[1] for column in cursor.fetchall()]
    table_columns_dict[table_name] = columns

cursor.close()
conn.close()

# Print the result
for table, columns in table_columns_dict.items():
    print(f"{table}: {', '.join(columns)}")


albums: id, name, album_group, album_type, release_date, popularity
artists: name, id, popularity, followers
audio_features: id, acousticness, analysis_url, danceability, duration, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature, valence
genres: id
r_albums_artists: album_id, artist_id
r_albums_tracks: album_id, track_id
r_artist_genre: genre_id, artist_id
r_track_artist: track_id, artist_id
tracks: id, disc_number, duration, explicit, audio_feature_id, name, preview_url, track_number, popularity, is_playable


In [17]:
select t.id, a.artist, ag.genre t.duration, t.audio_feature_id, t.popularity, af.acousticness, af.danceability, af.energy, af.instrumentalness, af.key,
af.liveness, af.loudness, af.speechiness, af.tempo, af.time_signature, af.valence
from tracks t
left join audio_features af
on t.audio_feature_id = af.id
left join r_track_artists a
on t.id = a.track_id
left join r_artist_genre ag
on a.artist_id = r_artist_genre.artist_id


SyntaxError: invalid syntax (2489885761.py, line 1)